In [1]:
from copy import deepcopy

import numpy as np
import pandas as pd

import datetime

from tensorflow.data import Dataset
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError

In [2]:
CONFIGS = {
    'data_path': '../data/',
    'model_path': '../model/',
    'model_name': 'multi_input',
    'model_type': 'cnn',
    
    'test_lenght': 24,
    'valid_start_index': 1992,
    'test_start_index': 2016,
    
    'batch_size': 64,
    'learning_rate': 1e-4,
    'epochs': 100,
    'es_patience': 10,
    
    'window_size': 7*24,
    'target_length': 3,
}

In [3]:
data_path = '../data/'

train_origin = pd.read_csv(data_path+'train.csv', encoding='cp949')

In [4]:
data = deepcopy(train_origin)

data.columns = [
    'num', 'date_time', 'target', 'temp', 'wind',
    'humid', 'rain', 'sun', 'non_elec_eq', 'sunlight_eq'
]

data = data.loc[data['num'] == 1, :]

print(f'data.shape: {data.shape}')

data.shape: (2040, 10)


In [5]:
def mk_time_data(data):
    
    new_data = data.copy()

    new_data['date_time'] = data['date_time'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H'))
    
    new_data['time_stamp'] = new_data['date_time'].apply(lambda x: x.timestamp())
    
    new_data['year'] = new_data['date_time'].apply(lambda x: x.year)
    new_data['month'] = new_data['date_time'].apply(lambda x: x.month)
    new_data['day'] = new_data['date_time'].apply(lambda x: x.day)
    
    new_data['hour'] = new_data['date_time'].apply(lambda x: x.hour)
    new_data['cos_hour'] = np.cos(2*np.pi*(new_data['hour']/24))
    new_data['sin_hour'] = np.sin(2*np.pi*(new_data['hour']/24))

    new_data['weekday'] = new_data['date_time'].apply(lambda x: x.weekday())
    new_data['cos_weekday'] = np.cos(2*np.pi*(new_data['weekday']/7))
    new_data['sin_weekday'] = np.sin(2*np.pi*(new_data['weekday']/7))
    
    new_data['is_holiday'] = 0
    new_data.loc[(new_data['weekday'] == 5) | (new_data['weekday'] == 6), 'is_holiday'] = 1
    new_data.loc[(new_data['month'] == 8) & (new_data['day'] == 17), 'is_holiday'] = 1
    
    return new_data

In [6]:
new_data = mk_time_data(data)

In [7]:
### target_date 정보도 쓰자

input_cols = [
    'temp', 'wind', 'humid', 'rain', 'sun', 'non_elec_eq',
    'sunlight_eq', 'time_stamp', 'cos_hour', 'sin_hour',
    'cos_weekday', 'sin_weekday', 'is_holiday', 'target',
]
target_cols = ['target']

CONFIGS['input_cols'] = input_cols
CONFIGS['target_cols'] = target_cols

In [8]:
def mk_dataset(data, CONFIGS, shuffle=False):
    
    X = data[CONFIGS['input_cols']][:-CONFIGS['target_length']]
    y = data[CONFIGS['target_cols']][CONFIGS['window_size']:]
    
    X_ds = Dataset.from_tensor_slices(X)
    X_ds = X_ds.window(CONFIGS['window_size'], shift=1, drop_remainder=True)
    X_ds = X_ds.flat_map(lambda x: x).batch(CONFIGS['window_size'])
    
    y_ds = Dataset.from_tensor_slices(y)
    y_ds = y_ds.window(CONFIGS['target_length'], shift=1, drop_remainder=True)
    y_ds = y_ds.flat_map(lambda x: x).batch(CONFIGS['target_length'])
    
    ds = Dataset.zip((X_ds, y_ds))
    if shuffle:
        ds = ds.shuffle(512)
    ds = ds.batch(CONFIGS['batch_size']).cache().prefetch(2)
    
    return ds

In [9]:
train = new_data.loc[:CONFIGS['valid_start_index'], :]
valid = new_data.loc[CONFIGS['valid_start_index']-CONFIGS['window_size']:CONFIGS['test_start_index'], :]
test = new_data.loc[CONFIGS['test_start_index']-CONFIGS['window_size']:, :]

In [10]:
train_ds = mk_dataset(train, CONFIGS, shuffle=True)
valid_ds = mk_dataset(valid, CONFIGS)
test_ds = mk_dataset(test, CONFIGS)

In [11]:
# flatten, cnn, lstm, attention 추가
# 모델 class 형태로 변형

def set_model(CONFIGS, model_name=None, print_summary=False):
    
    inputs = Input(batch_shape=(
        None, CONFIGS['window_size'], len(CONFIGS['input_cols'])
    ), name='inputs')
    
    if CONFIGS['model_type'] == 'flatten':
        flatten = Flatten(name='flatten')(inputs)
    elif CONFIGS['model_type'] == 'cnn':
        reshape = Reshape(target_shape=(
            CONFIGS['window_size'], len(CONFIGS['input_cols']), 1
        ), name='reshape')(inputs)
        conv_0 = Conv2D(8, (3, 1), strides=(2, 1), activation='relu', name='conv_0')(reshape)
        pool_0 = MaxPool2D((2, 1), name='pool_0')(conv_0)
        conv_1 = Conv2D(16, (3, 1), strides=(2, 1), activation='relu', name='conv_1')(pool_0)
        pool_1 = MaxPool2D((2, 1), name='pool_1')(conv_1)
        flatten = Flatten(name='flatten')(pool_1)
    elif CONFIGS['model_type'] == 'bilstm':
        bilstm_0 = Bidirectional(LSTM(
            16, return_sequences=True, recurrent_dropout=0.3, activation='relu'
        ), name='bilstm_0')(inputs)
        bilstm_1 = Bidirectional(LSTM(
            32, recurrent_dropout=0.3, activation='relu'
        ), name='bilstm_1')(bilstm_0)
        flatten = Flatten(name='flatten')(bilstm_1)
#     elif CONFIGS['model_type'] == 'attention':
        
    dense_0 = Dense(64, activation='relu', name='dense_0')(flatten)
    dense_1 = Dense(32, activation='relu', name='dense_1')(dense_0)
    outputs = Dense(CONFIGS['target_length'], name='outputs')(dense_1)
    
    if not model_name:
        model_name = CONFIGS['model_name']
    
    model = Model(
        inputs, outputs,
        name = model_name
    )
    
    optimizer = Adam(learning_rate=CONFIGS['learning_rate'])
    model.compile(
        loss = MeanSquaredError(),
        optimizer = optimizer,
    )
    
    if print_summary:
        model.summary()
    
    return model

In [12]:
model = set_model(CONFIGS, print_summary=True)
# model = set_model(CONFIGS, 'bilstm', print_summary=True)

Model: "multi_input"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 168, 14)]         0         
_________________________________________________________________
reshape (Reshape)            (None, 168, 14, 1)        0         
_________________________________________________________________
conv_0 (Conv2D)              (None, 83, 14, 8)         32        
_________________________________________________________________
pool_0 (MaxPooling2D)        (None, 41, 14, 8)         0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 20, 14, 16)        400       
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 10, 14, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 2240)              

In [13]:
def train_model(model, train_ds, valid_ds, CONFIGS):
    
    early_stop = EarlyStopping(
        patience=CONFIGS['es_patience']
    )
    save_best_only = ModelCheckpoint(
        filepath = f'{CONFIGS["model_path"]}{CONFIGS["model_name"]}.h5',
        monitor = 'val_loss',
        save_best_only = True,
        save_weights_only = True
    )
    
    history = model.fit(
        train_ds,
        batch_size = CONFIGS['batch_size'],
        epochs = CONFIGS['epochs'],
        validation_data = valid_ds,
        callbacks = [
            early_stop,
            save_best_only,
        ]
    )
    
    return history

In [14]:
history = train_model(model, train_ds, valid_ds, CONFIGS)

Epoch 1/100
29/29 [==============================] - 2s 9ms/step - loss: 7818196287488.0000 - val_loss: 743945797632.0000
Epoch 2/100
29/29 [==============================] - 0s 3ms/step - loss: 340416004096.0000 - val_loss: 1158880256.0000
Epoch 3/100
29/29 [==============================] - 0s 4ms/step - loss: 15383173120.0000 - val_loss: 2532577792.0000
Epoch 4/100
29/29 [==============================] - 0s 3ms/step - loss: 949856576.0000 - val_loss: 22795988.0000
Epoch 5/100
29/29 [==============================] - 0s 3ms/step - loss: 40000508.0000 - val_loss: 1314307.6250
Epoch 6/100
29/29 [==============================] - 0s 3ms/step - loss: 2010272.0000 - val_loss: 277754.5000
Epoch 7/100
29/29 [==============================] - 0s 3ms/step - loss: 123320.3125 - val_loss: 23086.6465
Epoch 8/100
29/29 [==============================] - 0s 3ms/step - loss: 11847.4141 - val_loss: 7186.3330
Epoch 9/100
29/29 [==============================] - 0s 3ms/step - loss: 8486.5518 - val_lo

In [15]:
best_model = set_model(CONFIGS, model_name='best_'+CONFIGS['model_name'])
best_model.load_weights(f'{CONFIGS["model_path"]}{CONFIGS["model_name"]}.h5')

In [16]:
y_train_pred = best_model.predict(train_ds)
y_valid_pred = best_model.predict(valid_ds)
y_test_pred = best_model.predict(test_ds)

In [17]:
train_loss = best_model.evaluate(train_ds, verbose=0)
valid_loss = best_model.evaluate(valid_ds, verbose=0)
test_loss = best_model.evaluate(test_ds, verbose=0)

print(f'train_loss: {train_loss}')
print(f'valid_loss: {valid_loss}')
print(f'test_loss: {test_loss}')

train_loss: 9082.861328125
valid_loss: 5577.3955078125
test_loss: 13567.5966796875
